# Neural Machine Translation by Jointly Learning to Align and Translate

In this third notebook on sequence-to-sequence models using PyTorch and TorchText, we'll be implementing the model from [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473). This model achives our best perplexity yet, ~27 compared to ~34 for the previous model.

## Introduction

Here is the general encoder-decoder model:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq1.png?raw=1)

In the previous model, our architecture was set-up in a way to reduce "information compression" by explicitly passing the context vector, $z$, to the decoder at every time-step and by passing both the context vector and embedded input word, $d(y_t)$, along with the hidden state, $s_t$, to the linear layer, $f$, to make a prediction.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq7.png?raw=1)

Even though we have reduced some of this compression, our context vector still needs to contain all of the information about the source sentence. The model implemented in this notebook avoids this compression by allowing the decoder to look at the entire source sentence (via its hidden states) at each decoding step! How does it do this? It uses *attention*. 

Attention works by first, calculating an attention vector, $a$, that is the length of the source sentence. The attention vector has the property that each element is between 0 and 1, and the entire vector sums to 1. We then calculate a weighted sum of our source sentence hidden states, $H$, to get a weighted source vector, $w$. 

$$w = \sum_{i}a_ih_i$$

We calculate a new weighted source vector every time-step when decoding, using it as input to our decoder RNN as well as the linear layer to make a prediction. We'll explain how to do all of this during the session.


## Import Statements

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import Multi30k
from torchtext import data
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time
import json
from tqdm import tqdm as tqdm

Set the random seeds for reproducability.

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
# downloading spacy english
# %%bash
# python -m spacy download en

## Previewing the Dataset

Link to the dataset https://rajpurkar.github.io/SQuAD-explorer/

In [4]:
with open("../datasets/Question-Answer Datasets for Chatbot Training/train-v2.0.json") as f:
    squad_data = json.load(f)

In [5]:
squad_data["data"][0]["paragraphs"][0]["qas"]

[{'question': 'When did Beyonce start becoming popular?',
  'id': '56be85543aeaaa14008c9063',
  'answers': [{'text': 'in the late 1990s', 'answer_start': 269}],
  'is_impossible': False},
 {'question': 'What areas did Beyonce compete in when she was growing up?',
  'id': '56be85543aeaaa14008c9065',
  'answers': [{'text': 'singing and dancing', 'answer_start': 207}],
  'is_impossible': False},
 {'question': "When did Beyonce leave Destiny's Child and become a solo singer?",
  'id': '56be85543aeaaa14008c9066',
  'answers': [{'text': '2003', 'answer_start': 526}],
  'is_impossible': False},
 {'question': 'In what city and state did Beyonce  grow up? ',
  'id': '56bf6b0f3aeaaa14008c9601',
  'answers': [{'text': 'Houston, Texas', 'answer_start': 166}],
  'is_impossible': False},
 {'question': 'In which decade did Beyonce become famous?',
  'id': '56bf6b0f3aeaaa14008c9602',
  'answers': [{'text': 'late 1990s', 'answer_start': 276}],
  'is_impossible': False},
 {'question': 'In what R&B group

In [6]:
squad_data["data"][0]["paragraphs"][0]["context"]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

## Preparing Data
### So converting the data above in a format having (context+question, answer) to be fed for training.
While combining context and question we will add one string "<mos>" to distinguish between context(paragraph) and question

In [7]:
quescontext_ans_pairs = []

for article in (tqdm(squad_data["data"])):
    for paragraph in article["paragraphs"]:
        context = paragraph["context"]
        for qa in paragraph["qas"]:
            quescontext = context + "<mos> " + qa["question"]
            if not qa["is_impossible"]:
                answer = qa["answers"][0]["text"]
            else:
                answer = "is impossilbe"
            quescontext_ans_pairs.append([quescontext, answer])

100%|██████████| 442/442 [00:00<00:00, 2035.23it/s]


In [8]:
len(quescontext_ans_pairs)

130319

In [9]:
quescontext_ans_pairs[0]

['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".<mos> When did Beyonce start becoming popular?',
 'in the late 1990s']

## Defining the Fields

Question and Answer are standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [10]:
Question = Field(sequential = True,
                      tokenize = 'spacy',
#                       batch_first =True,
                      include_lengths=True,
                      init_token = '<sos>',
                      eos_token = '<eos>',
                      lower = True)

Answer = Field(sequential = True,
                    tokenize ='spacy',
#                     batch_first =True,
                    include_lengths=True,
                    init_token = '<sos>',
                    eos_token = '<eos>',
                    lower = True)

Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the list quescontext_ans_pairs

In [11]:
# fields = [('questions', Question),('answers',Answer)]
fields = [('questions', Question),('answers',Question)]

In [12]:
example = [data.Example.fromlist([quescontext_ans_pairs[i][0],quescontext_ans_pairs[i][1]], fields) for i in tqdm(range(len(quescontext_ans_pairs)))] 

100%|██████████| 130319/130319 [02:00<00:00, 1078.14it/s]


## Removing stopwords

In [13]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwods')
stopwords = stopwords.words("english")

def remove_stopwords(word_list):
    return [word for word in word_list if word not in stopwords]

[nltk_data] Error loading stopwods: Package 'stopwods' not found in
[nltk_data]     index


In [14]:
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [15]:
len(stopwords)

179

In [16]:
for idx, i in tqdm(enumerate(example)):
    example[idx].questions = remove_stopwords(i.questions)

In [17]:
len(example)

130319

## Creating dataset

In [18]:
# creating dataset
squadDataset = data.Dataset(example, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [19]:
(train, valid, test) = squadDataset.split(split_ratio=[0.70, 0.15, 0.15], random_state=random.seed(SEED))

In [20]:
(len(train), len(valid), len(test))

(91223, 19548, 19548)

An example from the dataset:

In [21]:
vars(train.examples[10])

{'questions': ['another',
  'issue',
  'use',
  'hypopodium',
  'standing',
  'platform',
  'support',
  'feet',
  ',',
  'given',
  'hands',
  'may',
  'able',
  'support',
  'weight',
  '.',
  '17th',
  'century',
  'rasmus',
  'bartholin',
  'considered',
  'number',
  'analytical',
  'scenarios',
  'topic',
  '.',
  '20th',
  'century',
  ',',
  'forensic',
  'pathologist',
  'frederick',
  'zugibe',
  'performed',
  'number',
  'crucifixion',
  'experiments',
  'using',
  'ropes',
  'hang',
  'human',
  'subjects',
  'various',
  'angles',
  'hand',
  'positions',
  '.',
  'experiments',
  'support',
  'angled',
  'suspension',
  ',',
  'two',
  '-',
  'beamed',
  'cross',
  ',',
  'perhaps',
  'form',
  'foot',
  'support',
  ',',
  'given',
  'aufbinden',
  'form',
  'suspension',
  'straight',
  'stake',
  '(',
  'used',
  'nazis',
  'dachau',
  'concentration',
  'camp',
  'world',
  'war',
  'ii',
  ')',
  ',',
  'death',
  'comes',
  'rather',
  'quickly.<mos',
  '>',
  'sai

## Data Augmentation
copied from the repository of Woncoh1 https://github.com/woncoh1/END/blob/main/Assignment_Session_7_Sentiment_Analysis_using_LSTM_RNN.ipynb

In [22]:
# import libraries and prepare sentence text

import random
# for back translation
import google_trans_new
from google_trans_new import google_translator
# for word tokenization
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/deep/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Back Translation

In [23]:
# translate a sentence to a random language,
# and translate back to original language

def back_translate(sentence, p=0.1):
    # do nothing with probability of 1-p
    if random.uniform(0,1) > p:
        return sentence
    
    # combine tokenized sentence into one string
    sentence = " ".join(sentence)
    
    # instantiate translator
    translator = google_translator()
    
    # choose a target language
    available_langs = list(google_trans_new.LANGUAGES.keys())
    trans_lang = random.choice(available_langs)
    
    # translate to the target language
    translations = translator.translate(sentence, lang_tgt=trans_lang)
    
    # translate back to original language
    translations_en_random = translator.translate(translations, lang_src=trans_lang, lang_tgt="en")
    
    # select only one translation
    if len(translations_en_random) > 1:
        translations_en_random = translations_en_random[0]
        
    return word_tokenize(translations_en_random)

#### Random Deletion

In [24]:
# randomly delete words from a sentence with a given probability

def random_deletion(sentence, p=0.5): 
    # return if single word
    if len(sentence) == 1: 
        return sentence
    # delete words
    remaining = list(filter(lambda x: random.uniform(0,1) > p, sentence)) 
    # if nothing left, sample a random word
    if len(remaining) == 0: 
        return [random.choice(sentence)] 
    else:
        return remaining

#### Random Swap

In [25]:
# randomly swap a pair of words in a sentence for a given # of times

def random_swap(sentence, n=5): 
    if len(sentence) < 2:
        return sentence
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

#### Carry Out Data Augmentation

In [26]:
for example in tqdm(train.examples): 
    example.questions = back_translate(example.questions, p=0.01)
    example.questions = random_deletion(example.questions, p=0.1)
    example.questions = random_swap(example.questions, n=1)

100%|██████████| 91223/91223 [09:32<00:00, 159.31it/s] 


## Building Vocabulary

In [44]:
MAX_VOCAB_SIZE = 25000
''' Use max of 25000 frequently occuring words to create vocabulary. Use pretrained fasttext embedding. Embedding of words that exist in the data but not in the
dictionary are set using normal distribution.
'''
Question.build_vocab(train,
                     max_size = MAX_VOCAB_SIZE,
                     vectors='fasttext.simple.300d',
                     unk_init = torch.Tensor.normal_,
                     min_freq = 2)
Answer.build_vocab(train,
                   vectors='fasttext.simple.300d',
                   min_freq = 2)

In [45]:
# get the vocab instance
vocab = Question.vocab

In [46]:
embedding_vector = Question.vocab.vectors
embedding_vector.shape

torch.Size([25004, 300])

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [47]:
print('Size of context+question vocab : ', len(Question.vocab))
print('Size of answer vocab : ', len(Answer.vocab))
print('Top 10 words appeared in contex+question:', list(Question.vocab.freqs.most_common(10)))
print("Top 10 words appeared in answer:", list(Answer.vocab.freqs.most_common(10)))

Size of context+question vocab :  25004
Size of answer vocab :  4
Top 10 words appeared in contex+question: [(',', 624240), ('.', 325459), ('"', 104398), ('-', 102183), ('(', 89076), (')', 85255), ('>', 81566), ('?', 81362), ("'s", 67096), ('is', 31131)]
Top 10 words appeared in answer: []


## Create the iterators

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [32]:
BATCH_SIZE = 16
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train, valid, test),
                                                            batch_size = BATCH_SIZE, 
                                                            sort_key = lambda x: len(x.questions),
                                                            sort_within_batch=True,
                                                            device = device)

In [33]:
vars(Question.vocab)

{'freqs': Counter({'historical': 1632,
          'reveal': 185,
          'policing': 247,
          'agents': 420,
          'undertaken': 131,
          'cross': 1318,
          '-': 102183,
          'police': 2163,
          'missions': 325,
          'many': 19224,
          'years': 9706,
          '(': 89076,
          'deflem': 9,
          ',': 624240,
          ')': 85255,
          '.': 325459,
          '19th': 2675,
          'ww2': 5,
          'european': 5395,
          'agencies': 650,
          'undertook': 85,
          'border': 1206,
          'surveillance': 123,
          'concerns': 679,
          'anarchist': 20,
          'political': 5745,
          'notable': 1314,
          'example': 5204,
          'prussian': 840,
          'karl': 340,
          'marx': 147,
          'remained': 2028,
          'resident': 322,
          'london': 3596,
          'interests': 540,
          'public': 6321,
          'operation': 1217,
          'control': 4560,
       

Save the vocabulary for later use

In [34]:
import os, pickle
with open('question_tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Question.vocab.stoi, tokens)
# with open("answer_tokenizer.pkl", "wb") as tokens:
#     pickle.dump(Answer.vocab.stoi, tokens)

## Building the model

## Building the Seq2Seq Model

### Encoder

The encoder is similar to the previous one, with the multi-layer LSTM swapped for a single-layer GRU. We also don't pass the dropout as an argument to the GRU as that dropout is used between each layer of a multi-layered RNN. As we only have a single layer, PyTorch will display a warning if we try and use pass a dropout value to it.

Another thing to note about the GRU is that it only requires and returns a hidden state, there is no cell state like in the LSTM.

$$\begin{align*}
h_t &= \text{GRU}(e(x_t), h_{t-1})\\
(h_t, c_t) &= \text{LSTM}(e(x_t), h_{t-1}, c_{t-1})\\
h_t &= \text{RNN}(e(x_t), h_{t-1})
\end{align*}$$

From the equations above, it looks like the RNN and the GRU are identical. Inside the GRU, however, is a number of *gating mechanisms* that control the information flow in to and out of the hidden state (similar to an LSTM). Again, for more info, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) excellent post. 

The rest of the encoder should be very familar from the last session, it takes in a sequence, $X = \{x_1, x_2, ... , x_T\}$, passes it through the embedding layer, recurrently calculates hidden states, $H = \{h_1, h_2, ..., h_T\}$, and returns a context vector (the final hidden state), $z=h_T$.

$$h_t = \text{EncoderGRU}(e(x_t), h_{t-1})$$

This is identical to the encoder of the general seq2seq model, with all the "magic" happening inside the GRU (green).

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq5.png?raw=1)

In [35]:
class Encoder(nn.Module):
    '''
    encoder rakes ainpuyt and runs it through the lstm
    for example let's say the input size is [30,128],
    here 30 means 30 words/onehot and 128 means batch size
    
    first pass it through the embedding dim of let's say 512
    so the embedding output will bw [30,128,512]
    then pass this to lstm which will give you output and hidden cell state
    outputs wil have output at each time step so the first dim is sec_len
    hidden and cell is the final state output so they don't have src_len in the output 
    let's say hidden dim for lstm is 10
    outputs = [src_len,batch_size,hid_dim*n_layers]
    hidden = [n layers * n directions, batch size, hid dim] [2,128,10] 2 for 2 layer
    cell = [n layers * n directions, batch size, hid dim] [2,128,10] 2 for 2 layer
    
    '''
    
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layers = 1
        
        self.embedding = nn.Embedding(input_dim,emb_dim) #no dropout as only one layer!
        self.rnn = nn.GRU(emb_dim,hid_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        # src = [src_len, batch_size, emb_dim]
        embedded = self.dropout(self.embedding(src))
        
        # embedded = [src_len, batch_size, emb_dim]
        outputs, hidden = self.rnn(embedded)  #no cell state because we are using GRU
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        # outputs are always from top hidden layer
        
        return hidden
        

## Decoder

The decoder is where the implementation differs significantly from the previous model and we alleviate some of the information compression.

Instead of the GRU in the decoder taking just the embedded target token, $d(y_t)$ and the previous hidden state $s_{t-1}$ as inputs, it also takes the context vector $z$. 

$$s_t = \text{DecoderGRU}(d(y_t), s_{t-1}, z)$$

Note how this context vector, $z$, does not have a $t$ subscript, meaning we re-use the same context vector returned by the encoder for every time-step in the decoder. 

Before, we predicted the next token, $\hat{y}_{t+1}$, with the linear layer, $f$, only using the top-layer decoder hidden state at that time-step, $s_t$, as $\hat{y}_{t+1}=f(s_t^L)$. Now, we also pass the embedding of current token, $d(y_t)$ and the context vector, $z$ to the linear layer.

$$\hat{y}_{t+1} = f(d(y_t), s_t, z)$$

Thus, our decoder now looks something like this:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq6.png?raw=1)

Note, the initial hidden state, $s_0$, is still the context vector, $z$, so when generating the first token we are actually inputting two identical context vectors into the GRU.

How do these two changes reduce the information compression? Well, hypothetically the decoder hidden states, $s_t$, no longer need to contain information about the source sequence as it is always available as an input. Thus, it only needs to contain information about what tokens it has generated so far. The addition of $y_t$ to the linear layer also means this layer can directly see what the token is, without having to get this information from the hidden state. 

However, this hypothesis is just a hypothesis, it is impossible to determine how the model actually uses the information provided to it (don't listen to anyone that says differently). Nevertheless, it is a solid intuition and the results seem to indicate that this modifications are a good idea!

Within the implementation, we will pass $d(y_t)$ and $z$ to the GRU by concatenating them together, so the input dimensions to the GRU are now `emb_dim + hid_dim` (as context vector will be of size `hid_dim`). The linear layer will take $d(y_t), s_t$ and $z$ also by concatenating them together, hence the input dimensions are now `emb_dim + hid_dim*2`. We also don't pass a value of dropout to the GRU as it only uses a single layer.

`forward` now takes a `context` argument. Inside of `forward`, we concatenate $y_t$ and $z$ as `emb_con` before feeding to the GRU, and we concatenate $d(y_t)$, $s_t$ and $z$ together as `output` before feeding it through the linear layer to receive our predictions, $\hat{y}_{t+1}$.

In [36]:
class Decoder(nn.Module):
    '''
        Decoder takes an input and runs it through the lstm
        decoder runs one word of batch
        for example let's say the input size is [128]
        here 128 means a batch of 128 words lets say all the 0th index 128 words
        
        first pass it through embedding dim of let's say 512
        so the embedding output will be [1,batch_size,emb_dim][1,128,512]
        then pass this ti lstm which will give you output and hidden cell state 
        outputs will have output at each time step so the first dim is src_len
        but since decoder has only one word of batch the first dim is 0 here
        hidden and cell os the dinal state output so they don't have src_len in the output as usual
        let's say hidden dim for lstm is 10
        outputs = [src_len,btach_size,hid_dim*n_directions][1,128,10] 1 here because decoder runs one word
        hidden = [n layers * n directions, batch size, hid dim] [2,128,10] 2 for 2 layer, we are using 1 here
        cell = [n layers * n directions, batch size, hid dim] [2,128,10] 2 for 2 layer, we are using 1 here
    '''
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)

        self.fc_out = nn.Linear(emb_dim + hid_dim * 2,
                                output_dim)  # we are using same dim here

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, context):
        
        # input = [batch_size]
        # hidden = [n_layers*n_direction,batch_size,hid_dim]
        # context = [n_layers*n_direction,batch_size,hid_dim]
        # n layers and n directions in the decoder will both always be 1,therefore ?? why always, only in this case, for example if we use 2 layers it may change
        # hidden = [1, batch_size, hid_dim]
        # context = [1, batch_size, hid_dim]
        
        input = input.unsqueeze(0)
        # input = [1, batch_size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1,batch_size,emb_dim]
        
        emb_con = torch.cat((embedded, context),
                             dim=2)  # last dim is output dim
        # emb_con = [1,batch_size,emb_dim+hid_dim]

        output, hidden = self.rnn(emb_con, hidden)
        # output = [seq_len,batch_size,hid_dim*n_directions]
        # hidden = [n_layers * n_directions,batch_size,hid_dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), dim = 1)
        
        prediction = self.fc_out(output.squeeze(0))
        #prediction = [batch size, output dim]
        
        # print('--------- check the shapes inside Decoder forward ---------')
        # print('input shape {}, hidden shape {}, context shape {}'.format(input.shape,hidden.shape,context.shape))
        # print('embedded shape {},emb_con shape {},output shape {},hidden shape {},'.format(embedded.shape,emb_con.shape,output.shape,hidden.shape))
        # print('prediction shape {}'.format(prediction.shape))
        
        return prediction, hidden

## Seq2Seq Model

Putting the encoder and decoder together, we get:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq7.png?raw=1)

Again, in this implementation we need to ensure the hidden dimensions in both the encoder and the decoder are the same.

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive a `context` vector
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and the context vector, $z$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate (either the ground truth next token in the target sequence or the highest predicted next token)

In [37]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"

    def forward(self, src, trg, teacher_forcing_ratio=0.5):

        # src = [src_len,batch_size]
        # trg = [trg_len,batch_size]

        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time

        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        # tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(device)

        # last hidden state of the encoder is the context
        context = self.encoder(src)

        # context also used as the initial hidden state of the decoder
        hidden = context

        # first inout to the decoder is the <sos> tokens
        input = trg[0, :]

        # print('--------- check the shapes inside Seq2Seq forward ---------')
        # print('src shape {}, trg shape {}'.format(src.shape,trg.shape))
        # print('batch_size is {} , trg_len is {} , trg_vocab_size {} , outputs shape {} ,'.format(batch_size,trg_len,trg_vocab_size,outputs.shape))
        # print('context vector {}, '.format(context.shape))
        
        for t in range(1, trg_len):

            # insert input token embeddingm previous hidden state and the context state
            # recieve output tensor i.e predictions and the new hidden state
            # context/hidden dim is [1,batch_size,hid_dim_enc], 1 for 1 layer
            # output dim [batch_size,vocab_len]
            
            output, hidden = self.decoder(input, hidden, context)

            # place predictions in a tensor holding predictions for each token
            outputs[t] = output

            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio

            # get the highest prediction token from out predictions
            # top1 has the dim [batch_size,vocab_len]
            # each example/word output a vector of target vocab length and we are taking the index of that
            # out has trg_vocab size , let's say there are 1000 words and by taking index
            # we are saying select the word which has highest value
            top1 = output.argmax(
                1)

            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

## Training the Seq2Seq Model

We initialise our parameters, encoder, decoder and seq2seq model (placing it on the GPU if we have one). 

In [38]:
len(Question.vocab)

25004

In [ ]:
INPUT_DIM = len(Question.vocab)
OUTPUT_DIM = len(Answer.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

We use a simplified version of the weight initialization scheme used in the paper. Here, we will initialize all biases to zero and all weights from $\mathcal{N}(0, 0.01)$.

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Calculate the number of parameters

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

We create an optimizer.

In [ ]:
optimizer = optim.Adam(model.parameters())

We initialize the loss function.

In [ ]:
ANSWER_PAD_IDX = Question.vocab.stoi[Question.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = ANSWER_PAD_IDX)

We then create the training loop...

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src, src_length = batch.questions
        trg, trg_length = batch.answers
        

        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
        if(i % 1000 == 0):
            print(f"{i} steps are done")
        
    return epoch_loss / len(iterator)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src, src_length = batch.questions
            trg, trg_length = batch.answers

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Finally, define a timing function.

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best 


In [ ]:
N_EPOCHS = 20
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'squad-attention.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Finally, we test the model on the test set using these "best" parameters.

In [ ]:
model.load_state_dict(torch.load('squad-attention.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')